# Item 4: Know how to view deduced types.

##  Key idea:
 
    Code editors in IDEs often show the types of program entities (e.g.
    variables, parameters, functions, etc.) when you do something like hover
    your cursor over the entity.


In [3]:
#include <iostream>
#include <boost/type_index.hpp>
using boost::typeindex::type_id_with_cvr;

In [7]:
{
    const int theAnswer = 42;

    auto x = theAnswer;
    auto y = &theAnswer;
    
    std::cout << type_id_with_cvr<decltype(x)>().pretty_name() << '\n';
    std::cout << type_id_with_cvr<decltype(y)>().pretty_name() << '\n';
}

int
int const*


## Key idea:
 
    An effective way to get a compiler to show a type it has deduced is to use
    that type in a way that leads to compilation problems.  The error message
    reporting the problem is virtually sure to mention the type that's causing
    it.

In [9]:
template<typename T>  // declaration only for TD;
class TD;             // TD = "Type Displayer"

In [10]:
{
    const int theAnswer = 42;

    auto x = theAnswer;
    auto y = &theAnswer;

    TD<decltype(x)> xType;  // elicit errors containing
    TD<decltype(y)> yType;  // x's and y's types
}

input_line_18:6:21: error: implicit instantiation of undefined template 'TD<int>'
    TD<decltype(x)> xType;  // elicit errors containing
                    ^
input_line_17:2:7: note: template is declared here
class TD;             // TD = "Type Displayer"
      ^
input_line_18:7:21: error: implicit instantiation of undefined template 'TD<const int *>'
    TD<decltype(y)> yType;  // x's and y's types
                    ^
input_line_17:2:7: note: template is declared here
class TD;             // TD = "Type Displayer"
      ^


Interpreter Error: 

## Key idea:
 
    Calls to std::type_info::name are not guaranteed to return anything
    sensible, but implementations try to be helpful.  The level of helpfullnes
    varies.


In [11]:
#include <iostream>
#include <typeinfo>

{
    const int theAnswer = 42;

    auto x = theAnswer;
    auto y = &theAnswer;

    std::cout << typeid(x).name() << '\n';   // display types for
    std::cout << typeid(y).name() << '\n';   // x and y
}

i
PKi


## Key idea:
 
    The results of std::type_info::name are not reliable.
    Compiler diagnostics are a more dependable source of information about the
    results of type deduction.

In [12]:
#include <iostream>
#include <typeinfo>
#include <vector>

class Widget {};

In [13]:
template<typename T>                // template function to
void f(const T& param)              // be called
{
  using std::cout;
  cout << "T =     " << typeid(T).name() << '\n';     // show T
  cout << "param = " << typeid(param).name() << '\n'; // show
                                                      // param's
                                                      // type

  TD<T> TType;                    // elicit errors containing
  TD<decltype(param)> paramType;  // T's and param's types
}

In [14]:
std::vector<Widget> createVec()    // factory function
{
    std::vector<Widget> vw;
    Widget w;
    vw.push_back(w);
    return vw;
}

In [15]:
{
    
    const auto vw = createVec();        // init vw w/factory return
    
    if (!vw.empty()) {
      f(&vw[0]);                        // call f
      // ...
    }
}

input_line_23:9:9: error: implicit instantiation of undefined template 'TD<const Widget *>'
  TD<T> TType;                    // elicit errors containing
        ^
input_line_25:7:7: note: in instantiation of function template specialization 'f<const Widget *>' requested here
      f(&vw[0]);                        // call f
      ^
input_line_17:2:7: note: template is declared here
class TD;             // TD = "Type Displayer"
      ^
input_line_23:10:23: error: implicit instantiation of undefined template 'TD<const Widget *const &>'
  TD<decltype(param)> paramType;  // T's and param's types
                      ^
input_line_17:2:7: note: template is declared here
class TD;             // TD = "Type Displayer"
      ^


Interpreter Error: 

## Key idea:
 
    It is possible to produce accurate type information using Boost.TypeIndex.


In [16]:
#include <iostream>
#include <typeinfo>
#include <vector>

template<typename T>
void f1(const T& param)
{
  // show T
  std::cout << "T =     "
       << type_id_with_cvr<T>().pretty_name()
       << '\n';

  // show param's type
  std::cout << "param = "
       << type_id_with_cvr<decltype(param)>().pretty_name()
       << '\n';
}

In [17]:
std::vector<Widget> createVec1()    // factory function
{
    std::vector<Widget> vw;
    Widget w;
    vw.push_back(w);
    return vw;
}

In [26]:
{

    auto vw = createVec1();        // init vw w/factory return
    std::cout << type_id_with_cvr<decltype(vw)>().pretty_name() << '\n';
    //if (!vw.empty()) {
      f1(&vw[0]);                        // call f1
      // ...
    //}
}

std::__1::vector<Widget, std::__1::allocator<Widget> >
T =     Widget*
param = Widget* const&


                               Things to Remember
                               ------------------

* Deduced types can often be seen using IDE editors, compiler error messages,
  and the Boost TypeIndex library.

* The results of some tools may be neither helpful nor accurate, so an
  understanding of C++'s type deduction rules remains essential.

